In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
import tensorflow.keras
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


TensorFlow version:  2.1.0


In [2]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [3]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [4]:
def construct_network(dropout, neuronPct, neuronShrink,noisePct):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten())
    model.add(Dense(neuronCount))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(14, activation='sigmoid')) # Output
    return model

In [5]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train", class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev", class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)

** compute class weights from training data **
** class_weights **
[{0: 0.976060692178489, 1: 0.023939307821511}, {0: 0.9379028967906056, 1: 0.06209710320939444}, {0: 0.977507900874183, 1: 0.02249209912581691}, {0: 0.9801862148908839, 1: 0.01981378510911613}, {0: 0.9642020357560434, 1: 0.03579796424395662}, {0: 0.9663727015263743, 1: 0.033627298473625666}, {0: 0.8859702012473223, 1: 0.11402979875267771}, {0: 0.9586866934982315, 1: 0.04131330650176841}, {0: 0.9623146440112557, 1: 0.03768535598874437}, {0: 0.9298929992036218, 1: 0.07010700079637826}, {0: 0.9335352709009039, 1: 0.06646472909909606}, {0: 0.9021976306069932, 1: 0.09780236939300682}, {0: 0.9453965277787032, 1: 0.05460347222129675}, {1: 0.720226409263611, 0: 0.27977359073638897}]


In [6]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 21 **


In [7]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train.csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev.csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [8]:
def optimize_network(dropout,lr,neuronPct,neuronShrink,alpha,gamma,noisePct):
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    output_weights_path = os.path.join(output_dir,  str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)+"_"+str(noisePct)+"_"+output_weights_name)
    print(f"** set output weights path to: {output_weights_path} **")
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
    model = construct_network(dropout, neuronPct, neuronShrink,noisePct)
    
    #model.compile(loss=focal_loss(gamma=gamma,alpha=alpha), optimizer=SGD(lr=lr))
    optimizer = SGD(lr=initial_learning_rate)
    model.compile(optimizer=optimizer,loss=focal_loss(gamma=gamma,alpha=alpha))
    #lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
    #lookahead.inject(model) # add into model
    output_weights_path = os.path.join(output_dir,  str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)+"_"+str(noisePct)+"_"+output_weights_name)
    print(f"** set output weights path to: {output_weights_path} **")
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
    model = construct_network(dropout, neuronPct, neuronShrink,noisePct)
    
    #model.compile(loss=focal_loss(gamma=gamma,alpha=alpha), optimizer=SGD(lr=lr))
    optimizer = SGD(lr=initial_learning_rate)
    model.compile(optimizer=optimizer,loss=focal_loss(gamma=gamma,alpha=alpha))
    #lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
    #lookahead.inject(model) # add into model
    callbacks = [
            checkpoint,
            keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, pmatience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
        ]
    
    model.summary()
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,

            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=class_weights,
            workers=generator_workers,
            shuffle=False,
        )
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(dropout)+"_"+str(lr)+"_"+"_"+str(neuronPct)+"_"+str(neuronShrink)+"_"+str(noisePct)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    
    with open(test_log_path, "w") as f:
        for i in range(len(class_names)):
            try:
                score = roc_auc_score(y[:, i], y_hat[:, i])
                aurocs.append(score)
            except ValueError:
                score = 0
            f.write(f"{class_names[i]}: {score}\n")
        mean_auroc = float(np.mean(aurocs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}")
    

    print("iteration|auroc|alpha|dropout|gamma|learning_rate|neuronPct|neuronShrink|noisePct")
    tensorflow.keras.backend.clear_session()
    time_took = time.time() - start_time
    return mean_auroc

        
    
    model.summary()
    


In [9]:
import csv
import re
pbounds = {}
for key, val in csv.reader(open("new_params.csv")):
    a,b = val.split(", ")
    a = re.sub('[()]', '', a)
    b = re.sub('[()]', '', b)
    print(a)
    print(b)
    pbounds[str(key)] = float(a),float(b)

0.7297456293468533
0.7797456293468533
0.15672137551441198
0.20672137551441197
1.2700405014991505
1.3200405014991503
0.011526645750030312
0.03652664575003031
0.16978944765075252
0.2197894476507525
0.33033165284973015
0.3803316528497302
0.282563134185142
0.33256313418514205


In [10]:
pbounds

{'alpha': (0.7297456293468533, 0.7797456293468533),
 'dropout': (0.15672137551441198, 0.20672137551441197),
 'gamma': (1.2700405014991505, 1.3200405014991503),
 'lr': (0.011526645750030312, 0.03652664575003031),
 'neuronPct': (0.16978944765075252, 0.2197894476507525),
 'neuronShrink': (0.33033165284973015, 0.3803316528497302),
 'noisePct': (0.282563134185142, 0.33256313418514205)}

In [11]:
from new_bayes_opt.bayesian_optimization import BayesianOptimization
import time

optimizer = BayesianOptimization(
    f=optimize_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1,
)

Kunci ['alpha', 'dropout', 'gamma', 'lr', 'neuronPct', 'neuronShrink', 'noisePct']
Bound  [[0.72974563 0.77974563]
 [0.15672138 0.20672138]
 [1.2700405  1.3200405 ]
 [0.01152665 0.03652665]
 [0.16978945 0.21978945]
 [0.33033165 0.38033165]
 [0.28256313 0.33256313]]


In [12]:
from bayes_opt.util import load_logs#from bayes_opt.util import load_logsm

#load_logs(new_optimizer, logs=["./noise10_1.json"])
#print("New optimizer is now aware of {} points.".format(len(new_optimizer.space)))

load_logs(optimizer, logs=["./noise12_1.json"])
print("New optimizer is now aware of {} points.".format(len(optimizer.space)))

from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
new_logger = JSONLogger(path="./noise12_2.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, new_logger)

Konten (1.125533007053861, 0.15804867401632372, 1.0003431244520347, 0.010813995435791039, 0.19073377945408557, 0.35092338594768796, 0.1558780634133013)
Konten (1.125533007053861, 0.15804867401632372, 1.0003431244520347, 0.010813995435791039, 0.19073377945408557, 0.35092338594768796, 0.1558780634133013)
Konten (1.0183410905645716, 0.1095151211346005, 2.616450202010071, 0.011515167086419769, 0.1934260975019838, 0.35204452249731516, 0.3634352309172837)
Konten (1.0183410905645716, 0.1095151211346005, 2.616450202010071, 0.011515167086419769, 0.1934260975019838, 0.35204452249731516, 0.3634352309172837)
Konten (0.5410813897968892, 0.15057012652676036, 2.251914407101381, 0.01235213897067451, 0.19070193469297617, 0.35198101489084876, 0.34022337060266106)
Konten (0.5410813897968892, 0.15057012652676036, 2.251914407101381, 0.01235213897067451, 0.19070193469297617, 0.35198101489084876, 0.34022337060266106)
Konten (1.9523923635790963, 0.09701362672388644, 3.0769678470079422, 0.01425833491377623, 0.

In [13]:
def convert_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [14]:
start_time = time.time()
optimizer.maximize(init_points=5,acq="weightedei", n_iter=2, omega=0.1)
time_took = time.time() - start_time

Isine x [0.75059673 0.1927376  1.27004622 0.01908496 0.17712724 0.33494858
 0.29187614]
Konten (0.750596729581982, 0.19273760018651986, 1.2700462202400178, 0.019084960065826308, 0.17712724219160816, 0.33494858258817006, 0.29187614475402557)
Masuk
** set output weights path to: ./experiments/0.19273760018651986_0.019084960065826308__0.17712724219160816_0.33494858258817006_0.29187614475402557_weights.h5 **
** set output weights path to: ./experiments/0.19273760018651986_0.019084960065826308__0.17712724219160816_0.33494858258817006_0.29187614475402557_weights.h5 **
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 296)               454952    
_________________________________________________________________
gaussian_nois

Epoch 3/7
75/76 [============================>.] - ETA: 0s - loss: 406.0761
Epoch 00003: val_loss improved from 439.67400 to 437.86410, saving model to ./experiments/0.17655974922594547_0.022006508610112684__0.20405042267059048_0.340554265336306_0.3264690060046893_weights.h5
76/76 [==============================] - 23s 298ms/step - loss: 405.5690 - val_loss: 437.8641
Epoch 4/7
75/76 [============================>.] - ETA: 0s - loss: 403.3283
Epoch 00004: val_loss improved from 437.86410 to 437.31690, saving model to ./experiments/0.17655974922594547_0.022006508610112684__0.20405042267059048_0.340554265336306_0.3264690060046893_weights.h5
76/76 [==============================] - 23s 299ms/step - loss: 402.9336 - val_loss: 437.3169
Epoch 5/7
75/76 [============================>.] - ETA: 0s - loss: 400.4683
Epoch 00005: val_loss improved from 437.31690 to 436.43507, saving model to ./experiments/0.17655974922594547_0.022006508610112684__0.20405042267059048_0.340554265336306_0.326469006004

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 76 steps, validate for 10 steps
Epoch 1/7
75/76 [============================>.] - ETA: 0s - loss: 449.3598
Epoch 00001: val_loss improved from inf to 422.25827, saving model to ./experiments/0.17239258442237412_0.03343637455743127__0.21451978082594486_0.33458386341821905_0.28451587334678613_weights.h5
76/76 [==============================] - 23s 300ms/step - loss: 448.4001 - val_loss: 422.2583
Epoch 2/7
75/76 [============================>.] - ETA: 0s - loss: 396.3787
Epoch 00002: val_loss improved from 422.25827 to 419.93888, saving model to ./experiments/0.17239258442237412_0.03343637455743127__0.21451978082594486_0.33458386341821905_0.28451587334678613_weights.h5
76/76 [==============================] - 23s 297ms/step - loss: 395.9893 - val_loss: 419.9389
Epoch 3/7
75/76 [============================>.] - ETA: 0s - loss: 389.9197
Epoch 00003: val_loss improved from 419.93888 to 419.58265, saving model to ./experiments/0.17

** set output weights path to: ./experiments/0.15672137551441198_0.03652664575003031__0.2197894476507525_0.3803316528497302_0.282563134185142_weights.h5 **
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 417)               640929    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 417)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 417)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 417)               0         
_________________________________________________________________
dense_3 (Dense)              (

Epoch 6/7
75/76 [============================>.] - ETA: 0s - loss: 388.7100
Epoch 00006: val_loss improved from 426.88263 to 426.59942, saving model to ./experiments/0.16219725174960964_0.030732651444174162__0.20414640631377562_0.336980729886974_0.2926593652360386_weights.h5
76/76 [==============================] - 23s 297ms/step - loss: 388.2328 - val_loss: 426.5994
Epoch 7/7
75/76 [============================>.] - ETA: 0s - loss: 387.2755
Epoch 00007: val_loss did not improve from 426.59942
21/21 [==============================] - 5s 260ms/step
** write log to ./experiments/0.16219725174960964_0.030732651444174162__0.20414640631377562_0.336980729886974_0.2926593652360386_test.log **
mean auroc: 0.8584102017902405
iteration|auroc|alpha|dropout|gamma|learning_rate|neuronPct|neuronShrink|noisePct
Konten (0.7680049425028138, 0.16219725174960964, 1.2961395219010874, 0.030732651444174162, 0.20414640631377562, 0.336980729886974, 0.2926593652360386)
Konten (0.7680049425028138, 0.16219725174

In [15]:
print(f"Total runtime: {convert_string(time_took)}")
print(optimizer.max)

Total runtime: 0:19:18.23
{'target': 0.8584571381663927, 'params': {'alpha': 0.7297456293468533, 'dropout': 0.15672137551441198, 'gamma': 1.2700405014991505, 'lr': 0.03652664575003031, 'neuronPct': 0.2197894476507525, 'neuronShrink': 0.3803316528497302, 'noisePct': 0.282563134185142}}
